LET'S WORK THROUGH ANALYZING THIS SOCCER DATA USING JOIN AND MANIPULATING DATA

- In a soccer league, points are assigned to teams based on the result of a game. Here, let's assume that 3 points are awarded for a win, 1 for a tie, and 0 for a defeat. Use the `match` table to calculate the running total of points earned by the team "Chelsea" (team id 8455) in the season "2014/2015".
- The final output should have the match date, the points earned by Chelsea, and the running total.

In [1]:
SELECT 
	date,
	-- Identify the home team as Barcelona or Real Madrid
	CASE WHEN hometeam_id = 8634 THEN 'FC Barcelona' 
                ELSE 'Real Madrid CF' END AS home,
    -- Identify the away team as Barcelona or Real Madrid
	CASE WHEN awayteam_id = 8634 THEN 'FC Barcelona' 
        ELSE 'Real Madrid CF' END AS away
FROM soccer.match
WHERE (awayteam_id = 8634 OR hometeam_id = 8634)
      AND (awayteam_id = 8633 OR hometeam_id = 8633);

,date,home,away
0,2011-12-10 00:00:00+00:00,Real Madrid CF,FC Barcelona
1,2012-04-21 00:00:00+00:00,FC Barcelona,Real Madrid CF
2,2013-03-02 00:00:00+00:00,Real Madrid CF,FC Barcelona
3,2012-10-07 00:00:00+00:00,FC Barcelona,Real Madrid CF
4,2013-10-26 00:00:00+00:00,FC Barcelona,Real Madrid CF
5,2014-03-23 00:00:00+00:00,Real Madrid CF,FC Barcelona
6,2015-03-22 00:00:00+00:00,FC Barcelona,Real Madrid CF
7,2014-10-25 00:00:00+00:00,Real Madrid CF,FC Barcelona


To calculate a SUM of logical values (i.e., TRUE/FALSE) directly. In SQL, you have to convert these values into 1 and 0 before calculating a sum. This can be done using a CASE statement.

In [1]:
SELECT
	-- Select country name and the count match IDs
    c.name AS country_name,
    COUNT(sub.id) AS matches
FROM soccer.country AS c
-- Inner join the subquery onto country
-- Select the country id and match id columns
INNER JOIN (SELECT country_id, id
           FROM soccer.match
           -- Filter the subquery by matches with 10+ goals
           WHERE (home_goal + away_goal) >= 10) AS sub
ON c.id= sub.country_id
GROUP BY country_name;

,country_name,matches
0,Netherlands,1
1,Spain,4
2,Germany,1
3,England,3


From the previous exercise, you found that England, Netherlands, Germany and Spain were the only countries that had matches in the database where 10 or more goals were scored overall. 
Let's find out some more details about those matches -- when they were played, during which seasons, and how many of the goals were home versus away goals.

In [2]:
SELECT
	-- Select country, date, home, and away goals from the subquery
    country,
    date,
    home_goal,
    away_goal
FROM 
	-- Select country name, date, home_goal, away_goal, and total goals in the subquery
	(SELECT c.name AS country, 
     	    m.date, 
     		m.home_goal, 
     		m.away_goal,
           (m.home_goal + m.away_goal) AS total_goals
    FROM soccer.match AS m
    LEFT JOIN soccer.country AS c
    ON m.country_id = c.id) AS subq
-- Filter by total goals scored in the main query
WHERE total_goals >= 10;

,country,date,home_goal,away_goal
0,England,2011-08-28 00:00:00+00:00,8,2
1,England,2012-12-29 00:00:00+00:00,7,3
2,England,2013-05-19 00:00:00+00:00,5,5
3,Germany,2013-03-30 00:00:00+00:00,9,2
4,Netherlands,2011-11-06 00:00:00+00:00,6,4
5,Spain,2013-10-30 00:00:00+00:00,7,3
6,Spain,2015-04-05 00:00:00+00:00,9,1
7,Spain,2015-05-23 00:00:00+00:00,7,3
8,Spain,2014-09-20 00:00:00+00:00,2,8


I will construct a query that calculates the average number of goals per match in each country's league.

In [3]:
SELECT 
	l.name AS league,
    -- Select and round the league's total goals
    ROUND(AVG(m.home_goal + m.away_goal), 2) AS avg_goals,
    -- Select & round the average total goals for the season
    (SELECT ROUND (AVG (home_goal + away_goal), 2) 
     FROM soccer.match
     WHERE season = '2013/2014') AS overall_avg
FROM soccer.league AS l
LEFT JOIN soccer.match AS m
ON l.country_id = m.country_id
-- Filter for the 2013/2014 season
WHERE season = '2013/2014'
GROUP BY l.name;

,league,avg_goals,overall_avg
0,Switzerland Super League,2.89,2.77
1,Poland Ekstraklasa,2.64,2.77
2,Netherlands Eredivisie,3.20,2.77
3,Scotland Premier League,2.75,2.77
4,France Ligue 1,2.46,2.77
5,Spain LIGA BBVA,2.75,2.77
6,Germany 1. Bundesliga,3.16,2.77
7,Italy Serie A,2.72,2.77
8,Portugal Liga ZON Sagres,2.37,2.77
9,England Premier League,2.77,2.77


I added a column that directly compares these values by subtracting the overall average from the subquery.

In [1]:
SELECT
	-- Select the league name and average goals scored
	l.name AS league,
	ROUND(AVG(m.home_goal + m.away_goal),2) AS avg_goals,
    -- Subtract the overall average from the league average
	ROUND(AVG(m.home_goal + m.away_goal) -
		(SELECT AVG (home_goal + away_goal)
		 FROM soccer.match 
         WHERE season = '2013/2014'),2) AS diff
FROM soccer.league AS l
LEFT JOIN soccer.match AS m
ON l.country_id = m.country_id
-- Only include 2013/2014 results
WHERE season = '2013/2014'
GROUP BY l.name;

,league,avg_goals,diff
0,Switzerland Super League,2.89,0.12
1,Poland Ekstraklasa,2.64,-0.13
2,Netherlands Eredivisie,3.20,0.43
3,Scotland Premier League,2.75,-0.02
4,France Ligue 1,2.46,-0.31
5,Spain LIGA BBVA,2.75,-0.02
6,Germany 1. Bundesliga,3.16,0.39
7,Italy Serie A,2.72,-0.04
8,Portugal Liga ZON Sagres,2.37,-0.40
9,England Premier League,2.77,0.00


In soccer leagues, games are played at different stages. Winning teams progress from one stage to the next, until they reach the final stage. In each stage, the stakes become higher than the previous one. The match table includes data about the different stages that each match took place in.

In this exercise, your query will extract data examining the average goals scored in each stage of a match. Does the average number of goals scored change as the stakes get higher from one stage to the next?

In [1]:
SELECT 
	-- Select the stage and average goals for each stage
	m.stage,
    ROUND(AVG(m.home_goal + m.away_goal),2) AS avg_goals,
    -- Select the average overall goals for the 2012/2013 season
    ROUND((SELECT AVG(home_goal + away_goal) 
           FROM soccer.match 
           WHERE season = '2012/2013'),2) AS overall
FROM soccer.match AS m
-- Filter for the 2012/2013 season
WHERE m.season = '2012/2013'
-- Group by stage
GROUP BY m.stage;

,stage,avg_goals,overall
0,29,2.63,2.77
1,4,2.80,2.77
2,34,2.68,2.77
3,32,2.75,2.77
4,9,2.70,2.77
5,7,2.69,2.77
6,10,2.96,2.77
7,35,2.43,2.77
8,38,3.17,2.77
9,15,2.76,2.77


A subquery to extract a list of stages where the average home goals in a stage is higher than the overall average for home goals in a match.

In [1]:
SELECT 
	-- Select the stage and average goals from the subquery
	s.stage,
	ROUND(s.avg_goals,2) AS avg_goals
FROM 
	-- Select the stage and average goals in 2012/2013
	(SELECT
		 stage,
         AVG (home_goal + away_goal) AS avg_goals
	 FROM soccer.match
	 WHERE season = '2012/2013'
	 GROUP BY stage) AS s
WHERE 
	-- Filter the main query using the subquery
	s.avg_goals > (SELECT AVG (home_goal + away_goal) 
                    FROM soccer.match WHERE season = '2012/2013');

,stage,avg_goals
0,4,2.80
1,10,2.96
2,38,3.17
3,6,2.78
4,12,3.23
5,36,2.90
6,31,3.06
7,30,2.87
8,21,2.90
9,3,2.83


what was the highest scoring match for each country, in each season?

In [1]:
SELECT 
	-- Select country ID, date, home, and away goals from match
	main.country_id,
    date,
    main.home_goal,
    main.away_goal
FROM soccer.match AS main
WHERE 
	-- Filter for matches with the highest number of goals scored
	(home_goal + away_goal) = 
        (SELECT MAX(sub.home_goal + sub.away_goal)
         FROM soccer.match AS sub
         WHERE main.country_id = sub.country_id
               AND main.season = sub.season);

,country_id,date,home_goal,away_goal
0,1,2011-10-29 00:00:00+00:00,4,5
1,1,2012-11-17 00:00:00+00:00,2,6
2,1,2012-12-09 00:00:00+00:00,1,7
3,1,2013-01-19 00:00:00+00:00,2,6
4,1,2012-08-19 00:00:00+00:00,2,6
...,...,...,...,...
73,24558,2012-09-30 00:00:00+00:00,6,2
74,24558,2014-02-16 00:00:00+00:00,5,3
75,24558,2015-04-30 00:00:00+00:00,6,2
76,24558,2015-05-03 00:00:00+00:00,2,6


Query to examine the highest total number of goals in each season, overall, and during July across all seasons.

In [1]:
SELECT
	-- Select the season and max goals scored in a match
	season,
    MAX(home_goal + away_goal) AS max_goals,
    -- Select the overall max goals scored in a match
   (SELECT MAX(home_goal + away_goal) FROM soccer.match) AS overall_max_goals,
   -- Select the max number of goals scored in any match in July
   (SELECT MAX(home_goal+ away_goal) 
    FROM soccer.match
    WHERE id IN (
          SELECT id FROM soccer.match WHERE EXTRACT(MONTH FROM date) = 07)) AS july_max_goals
FROM soccer.match
GROUP BY season;

,season,max_goals,overall_max_goals,july_max_goals
0,2013/2014,10,11,7
1,2012/2013,11,11,7
2,2014/2015,10,11,7
3,2011/2012,10,11,7


In [1]:
SELECT
	c.name AS country,
    -- Calculate the average matches per season
	AVG(outer_s.matches) AS avg_seasonal_high_scores
FROM country AS c
-- Left join outer_s to country
LEFT JOIN (
  SELECT country_id, season,
         COUNT(id) AS matches
  FROM (
    SELECT country_id, season, id
	FROM match
	WHERE home_goal >= 5 OR away_goal >= 5) AS inner_s
  -- Close parentheses and alias the subquery
  GROUP BY country_id, season) AS outer_s
ON c.id = outer_s.country_id
GROUP BY country;

,country,avg_seasonal_high_scores
0,Portugal,7.500000
1,France,8.000000
2,Scotland,8.000000
3,Netherlands,21.000000
4,Spain,22.000000
5,Belgium,11.333333
6,Italy,8.750000
7,Germany,13.750000
8,England,15.000000
9,Switzerland,5.500000


In [ ]:
SELECT
	m.date,
    -- Get the home and away team names
    hometeam,
    awayteam,
    m.home_goal,
    m.away_goal
FROM match AS m

-- Join the home subquery to the match table
LEFT JOIN (
  SELECT match.id, team.team_long_name AS hometeam
  FROM match
  LEFT JOIN team
  ON match.hometeam_id = team.team_api_id) AS home
ON home.id  = m.id

-- Join the away subquery to the match table
LEFT JOIN (
  SELECT match.id, team.team_long_name AS awayteam
  FROM match
  LEFT JOIN team
  -- Get the away team ID in the subquery
  ON match.awayteam_ID = team.team_api_id) AS away
ON away.id = m.id;

How do you get both the home and away team names into one final query result?

In [ ]:
SELECT
    m.date,
    (SELECT team_long_name
     FROM team AS t
     WHERE t.team_api_id = m.hometeam_id) AS hometeam,
    -- Connect the team to the match table
    (SELECT team_long_name
     FROM team AS t
     WHERE t.team_api_id = m.awayteam_id) AS awayteam,
    -- Select home and away goals
     home_goal,
     away_goal
FROM match AS m;

CREATED LEAGUE RANKINGS

In [2]:
SELECT 
	-- Select the league name and average goals scored
	l.name AS league,
    AVG(m.home_goal + m.away_goal) AS avg_goals,
    -- Rank each league according to the average goals
    RANK () OVER(ORDER BY AVG(m.home_goal + m.away_goal)) AS league_rank
FROM soccer.league AS l
LEFT JOIN soccer.match AS m 
ON l.id = m.country_id
WHERE m.season = '2011/2012'
GROUP BY l.name
-- Order the query by the rank you created
ORDER BY avg_goals;

,league,avg_goals,league_rank
0,Poland Ekstraklasa,2.195833,1
1,France Ligue 1,2.515789,2
2,Italy Serie A,2.583799,3
3,Switzerland Super League,2.623457,4
4,Scotland Premier League,2.635965,5
5,Portugal Liga ZON Sagres,2.641667,6
6,Spain LIGA BBVA,2.763158,7
7,England Premier League,2.805263,8
8,Germany 1. Bundesliga,2.859477,9
9,Belgium Jupiler League,2.879167,10


In [3]:
SELECT 
	-- Select the league name and average goals scored
	l.name AS league,
    AVG( m.home_goal + m.away_goal) AS avg_goals,
    -- Rank leagues in descending order by average goals
    RANK() OVER(ORDER BY AVG(M.home_goal + m.away_goal) DESC) AS league_rank
FROM soccer.league AS l
LEFT JOIN soccer.match AS m 
ON l.id = m.country_id
WHERE m.season = '2011/2012'
GROUP BY l.name
-- Order the query by the rank you created
ORDER BY avg_goals;

,league,avg_goals,league_rank
0,Poland Ekstraklasa,2.195833,11
1,France Ligue 1,2.515789,10
2,Italy Serie A,2.583799,9
3,Switzerland Super League,2.623457,8
4,Scotland Premier League,2.635965,7
5,Portugal Liga ZON Sagres,2.641667,6
6,Spain LIGA BBVA,2.763158,5
7,England Premier League,2.805263,4
8,Germany 1. Bundesliga,2.859477,3
9,Belgium Jupiler League,2.879167,2


Where do you see more outliers? Are they Legia Warszawa's home or away games?

In [1]:
SELECT
	date,
	season,
	home_goal,
	away_goal,
	CASE WHEN hometeam_id = 8673 THEN 'home' 
		 ELSE 'away' END AS warsaw_location,
    -- Calculate the average goals scored partitioned by season
    AVG (home_goal) OVER(PARTITION BY season) AS season_homeavg,
    AVG (away_goal) OVER (PARTITION BY season) AS season_awayavg
FROM soccer.match
-- Filter the data set for Legia Warszawa matches only
WHERE 
	hometeam_id = 8673
    OR awayteam_id = 8673
ORDER BY (home_goal + away_goal) DESC;

,date,season,home_goal,away_goal,warsaw_location,season_homeavg,season_awayavg
0,2013-09-14 00:00:00+00:00,2013/2014,3,5,away,1.766667,1.233333
1,2014-09-13 00:00:00+00:00,2014/2015,4,3,home,1.566667,1.333333
2,2013-07-20 00:00:00+00:00,2013/2014,5,1,home,1.766667,1.233333
3,2013-10-20 00:00:00+00:00,2013/2014,4,1,home,1.766667,1.233333
4,2013-06-02 00:00:00+00:00,2012/2013,5,0,home,1.566667,1.133333
...,...,...,...,...,...,...,...
115,2013-05-30 00:00:00+00:00,2012/2013,0,0,away,1.566667,1.133333
116,2013-04-27 00:00:00+00:00,2012/2013,0,0,away,1.566667,1.133333
117,2013-03-02 00:00:00+00:00,2012/2013,0,0,home,1.566667,1.133333
118,2015-04-24 00:00:00+00:00,2014/2015,0,0,away,1.566667,1.333333


Calculate the average number home and away goals scored Legia Warszawa, and their opponents, partitioned by the month in each season.

In [2]:
SELECT 
	date,
	season,
	home_goal,
	away_goal,
	CASE WHEN hometeam_id = 8673 THEN 'home' 
         ELSE 'away' END AS warsaw_location,
	-- Calculate average goals partitioned by season and month
    AVG (home_goal) OVER(PARTITION BY season, 
         	EXTRACT(month FROM date)) AS season_mo_home,
    AVG(away_goal) OVER (PARTITION BY season, 
            EXTRACT(month FROM date)) AS season_mo_away
FROM soccer.match
WHERE 
	hometeam_id = 8673
    OR awayteam_id = 8673
ORDER BY (home_goal + away_goal) DESC;

,date,season,home_goal,away_goal,warsaw_location,season_mo_home,season_mo_away
0,2013-09-14 00:00:00+00:00,2013/2014,3,5,away,2.250000,2.500000
1,2014-09-13 00:00:00+00:00,2014/2015,4,3,home,2.000000,2.666667
2,2013-07-20 00:00:00+00:00,2013/2014,5,1,home,2.500000,2.000000
3,2014-08-09 00:00:00+00:00,2014/2015,5,0,home,2.000000,1.000000
4,2012-10-28 00:00:00+00:00,2012/2013,3,2,home,1.666667,2.000000
...,...,...,...,...,...,...,...
115,2011-10-30 00:00:00+00:00,2011/2012,0,0,away,1.000000,0.250000
116,2011-11-07 00:00:00+00:00,2011/2012,0,0,away,1.750000,0.000000
117,2015-02-22 00:00:00+00:00,2014/2015,0,0,away,0.500000,1.500000
118,2012-03-30 00:00:00+00:00,2011/2012,0,0,away,0.600000,0.400000


Your first task is to create the first query that filters for matches where Manchester United played as the home team. This will become a common table expression in a later exercise.

In [ ]:
SELECT 
    m.id, 
    t.team_long_name,
    -- Identify matches as home/away wins or ties
    CASE WHEN m.home_goal > m.away_goal THEN 'MU Win'
         WHEN m.home_goal < m.away_goal THEN 'MU Loss' 
         ELSE 'Tie' END AS outcome
FROM match AS m
-- Left join team on the home team ID and team API id
LEFT JOIN team AS t 
ON m.hometeam_id = t.team_api_id
WHERE 
    -- Filter for 2014/2015 and Manchester United as the home team
    m.season = '2014/2015'
    AND t.team_long_name = 'Manchester United';

In [ ]:
SELECT 
    m.id, 
    t.team_long_name,
    -- Identify matches as home/away wins or ties
    CASE WHEN m.home_goal > m.away_goal THEN 'MU Loss'
         WHEN m.home_goal < m.away_goal THEN 'MU Win' 
         ELSE 'Tie' END AS outcome
-- Join team table to the match table
FROM match AS m
LEFT JOIN team AS t 
ON m.awayteam_id = t.team_api_id
WHERE 
    -- Filter for 2014/2015 and Manchester United as the away team
    m.season = '2014/2015'
    AND t.team_long_name = 'Manchester United';

Continue building the query to extract all matches played by Manchester United in the 2014/2015 season.

In [ ]:
-- Set up the home team CTE
WITH home AS (
  SELECT m.id, t.team_long_name,
      CASE WHEN m.home_goal > m.away_goal THEN 'MU Win'
           WHEN m.home_goal < m.away_goal THEN 'MU Loss' 
           ELSE 'Tie' END AS outcome
  FROM match AS m
  LEFT JOIN team AS t ON m.hometeam_id = t.team_api_id),
-- Set up the away team CTE
away AS (
  SELECT m.id, t.team_long_name,
      CASE WHEN m.home_goal > m.away_goal THEN 'MU Loss'
           WHEN m.home_goal < m.away_goal THEN 'MU Win' 
           ELSE 'Tie' END AS outcome
  FROM match AS m
  LEFT JOIN team AS t ON m.awayteam_id = t.team_api_id)
-- Select team names, the date and goals
SELECT DISTINCT
    m.date,
    home.team_long_name AS home_team,
    away.team_long_name AS away_team,
    m.home_goal, m.away_goal
-- Join the CTEs onto the match table
FROM match AS m
LEFT JOIN home ON m.id = home.id
LEFT JOIN away ON m.id = away.id
WHERE m.season = '2014/2015'
      AND (home.team_long_name = 'Manchester United' 
           OR away.team_long_name = 'Manchester United');

Fantastic! You now have a result set that retrieves the match date, home team, away team, and the goals scored by each team. You have one final component of the question left -- how badly did Manchester United lose in each match?

In [ ]:
-- Set up the home team CTE
WITH home AS (
  SELECT m.id, t.team_long_name,
      CASE WHEN m.home_goal > m.away_goal THEN 'MU Win'
           WHEN m.home_goal < m.away_goal THEN 'MU Loss' 
           ELSE 'Tie' END AS outcome
  FROM match AS m
  LEFT JOIN team AS t ON m.hometeam_id = t.team_api_id),
-- Set up the away team CTE
away AS (
  SELECT m.id, t.team_long_name,
      CASE WHEN m.home_goal > m.away_goal THEN 'MU Loss'
           WHEN m.home_goal < m.away_goal THEN 'MU Win' 
           ELSE 'Tie' END AS outcome
  FROM match AS m
  LEFT JOIN team AS t ON m.awayteam_id = t.team_api_id)
-- Select columns and and rank the matches by goal difference
SELECT DISTINCT
    m.date,
    home.team_long_name AS home_team,
    away.team_long_name AS away_team,
    m.home_goal, m.away_goal,
    RANK() OVER(ORDER BY ABS(home_goal - away_goal) DESC) as match_rank
-- Join the CTEs onto the match table
FROM match AS m
LEFT JOIN home ON m.id = home.id
LEFT JOIN AWAY ON m.id = away.id
WHERE m.season = '2014/2015'
      AND ((home.team_long_name = 'Manchester United' AND home.outcome = 'MU Loss')
      OR (away.team_long_name = 'Manchester United' AND away.outcome = 'MU Loss'));